In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False,
                                               split_noise=True, n_fabricate_noise = 1600,
                                               split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm import *

In [4]:
graph_name = 'SpectrogramMultiLSTM'  # Specify graph class name

g_cnfg = Config('graph_02_dropout075_on_input')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

#g_cnfg.conv1_n_filters = 16
#g_cnfg.conv1_kernel_size = 5
#g_cnfg.conv1_mp_size = 5
#g_cnfg.conv1_mp_strides = 2

#g_cnfg.conv2_n_filters = 32
#g_cnfg.conv2_kernel_size = 3
#g_cnfg.conv2_mp_size = 3
#g_cnfg.conv2_mp_strides = 2

g_cnfg.lstm_state_sizes = [100, 200]
#g_cnfg.lstm_dropout = {'in': 'y', 'out': 'n', 'state': 'n'}

#g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.n_conv_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters
#g_cnfg.n_flat = g_cnfg.n_conv_flat + g_cnfg.lstm_state_sizes[-1]

g_cnfg.n_hidden1 = 50
#g_cnfg.n_hidden2 = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('run_05_input_melpower_spec')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label('_background_noise_')])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [5]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_02_dropout075_on_input
X_img_h: 128
X_img_w: 71
Y_vector_len: 31
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [100, 200]
n_hidden1: 50

name: run_05_input_melpower_spec
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 379
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramMultiLSTM_graph_02_dropout075_on_input_run_05_input_melpower_spec
Training starts @ 12/16/2017 08:09:59
Epoch 1 Step 1 Best model saved @ 12/16/2017 08:10:04 [Ave valid] 3.434
Epoch 1 Step 1 ends @ 12/16/2017 08:10:04 [Train] 3.434, 2.7% [Valid] 3.6% [Ave valid] 3.434
Epoch 1 Step 50 Best model saved @ 12/16/2017 08:10:13 [Ave valid] 3.432
Epoch 1 Step 150 Best model saved @ 12/16/2017 08:10:33 [Ave valid] 3.431
Epoch 1 Step 250 Best model saved @ 12/16/2017 08:10:52 [Ave va

Epoch 13 Step 4,750 Best model saved @ 12/16/2017 08:26:17 [Ave valid] 0.400
Epoch 13 Step 4,800 Best model saved @ 12/16/2017 08:26:27 [Ave valid] 0.398
Epoch 13 Step 4,850 Best model saved @ 12/16/2017 08:26:37 [Ave valid] 0.396
Epoch 13 Step 4,900 Best model saved @ 12/16/2017 08:26:46 [Ave valid] 0.375
Epoch 13 Step 4,927 Best model saved @ 12/16/2017 08:26:53 [Ave valid] 0.370
Epoch 13 Step 4,927 ends @ 12/16/2017 08:26:53 [Train] 0.032, 100.0% [Valid] 88.8% [Ave valid] 0.370
Epoch 14 Step 4,950 Best model saved @ 12/16/2017 08:27:00 [Ave valid] 0.370
Epoch 14 Step 5,000 Best model saved @ 12/16/2017 08:27:10 [Ave valid] 0.369
Epoch 14 Step 5,050 Best model saved @ 12/16/2017 08:27:20 [Ave valid] 0.366
Epoch 14 Step 5,200 Best model saved @ 12/16/2017 08:27:48 [Ave valid] 0.362
Epoch 14 Step 5,300 Best model saved @ 12/16/2017 08:28:08 [Ave valid] 0.356
Epoch 14 Step 5,306 ends @ 12/16/2017 08:28:12 [Train] 0.048, 100.0% [Valid] 85.8% [Ave valid] 0.369
Epoch 15 Step 5,550 Best mod

Epoch 41 Step 15,539 Best model saved @ 12/16/2017 09:02:52 [Ave valid] 0.199
Epoch 41 Step 15,539 ends @ 12/16/2017 09:02:52 [Train] 0.014, 100.0% [Valid] 93.9% [Ave valid] 0.199
Epoch 42 Step 15,650 Best model saved @ 12/16/2017 09:03:16 [Ave valid] 0.199
Epoch 42 Step 15,700 Best model saved @ 12/16/2017 09:03:26 [Ave valid] 0.198
Epoch 42 Step 15,750 Best model saved @ 12/16/2017 09:03:35 [Ave valid] 0.197
Epoch 42 Step 15,800 Best model saved @ 12/16/2017 09:03:45 [Ave valid] 0.196
Epoch 42 Step 15,900 Best model saved @ 12/16/2017 09:04:04 [Ave valid] 0.192
Epoch 42 Step 15,918 Best model saved @ 12/16/2017 09:04:09 [Ave valid] 0.192
Epoch 42 Step 15,918 ends @ 12/16/2017 09:04:09 [Train] 0.217, 92.9% [Valid] 94.7% [Ave valid] 0.192
Epoch 43 Step 15,950 Best model saved @ 12/16/2017 09:04:18 [Ave valid] 0.191
Epoch 43 Step 16,000 Best model saved @ 12/16/2017 09:04:27 [Ave valid] 0.190
Epoch 43 Step 16,050 Best model saved @ 12/16/2017 09:04:37 [Ave valid] 0.190
Epoch 43 Step 16,

Epoch 83 Step 31,457 ends @ 12/16/2017 09:55:57 [Train] 0.205, 92.9% [Valid] 93.7% [Ave valid] 0.171
Epoch 84 Step 31,836 ends @ 12/16/2017 09:57:12 [Train] 0.016, 100.0% [Valid] 94.2% [Ave valid] 0.165
Epoch 85 Step 32,200 Best model saved @ 12/16/2017 09:58:24 [Ave valid] 0.159
Epoch 85 Step 32,215 Best model saved @ 12/16/2017 09:58:29 [Ave valid] 0.159
Epoch 85 Step 32,215 ends @ 12/16/2017 09:58:29 [Train] 0.002, 100.0% [Valid] 95.8% [Ave valid] 0.159
Epoch 86 Step 32,594 ends @ 12/16/2017 09:59:42 [Train] 0.027, 100.0% [Valid] 94.4% [Ave valid] 0.168
Epoch 87 Step 32,950 Best model saved @ 12/16/2017 10:00:52 [Ave valid] 0.158
Epoch 87 Step 32,973 Best model saved @ 12/16/2017 10:00:58 [Ave valid] 0.156
Epoch 87 Step 32,973 ends @ 12/16/2017 10:00:58 [Train] 0.003, 100.0% [Valid] 96.1% [Ave valid] 0.156
Epoch 88 Step 33,000 Best model saved @ 12/16/2017 10:01:06 [Ave valid] 0.156
Epoch 88 Step 33,352 ends @ 12/16/2017 10:02:14 [Train] 1.169, 71.4% [Valid] 92.4% [Ave valid] 0.176


Epoch 143 Step 54,100 Hourly model saved @ 12/16/2017 11:10:15
Epoch 143 Step 54,197 ends @ 12/16/2017 11:10:33 [Train] 0.197, 92.9% [Valid] 96.4% [Ave valid] 0.146
Epoch 144 Step 54,576 ends @ 12/16/2017 11:11:49 [Train] 0.002, 100.0% [Valid] 96.2% [Ave valid] 0.146
Epoch 145 Step 54,955 ends @ 12/16/2017 11:13:04 [Train] 0.001, 100.0% [Valid] 96.2% [Ave valid] 0.146
Epoch 146 Step 55,334 ends @ 12/16/2017 11:14:17 [Train] 0.551, 92.9% [Valid] 96.3% [Ave valid] 0.154
Epoch 147 Step 55,713 ends @ 12/16/2017 11:15:32 [Train] 0.002, 100.0% [Valid] 96.2% [Ave valid] 0.149
Epoch 148 Step 56,092 ends @ 12/16/2017 11:16:44 [Train] 0.001, 100.0% [Valid] 95.9% [Ave valid] 0.152
Epoch 149 Step 56,471 ends @ 12/16/2017 11:18:00 [Train] 0.026, 100.0% [Valid] 95.9% [Ave valid] 0.151
Epoch 150 Step 56,850 ends @ 12/16/2017 11:19:12 [Train] 0.001, 100.0% [Valid] 95.9% [Ave valid] 0.150
Epoch 151 Step 57,229 ends @ 12/16/2017 11:20:25 [Train] 0.002, 100.0% [Valid] 91.4% [Ave valid] 0.175
Epoch 152 St

Epoch 215 Step 81,485 ends @ 12/16/2017 12:39:22 [Train] 0.000, 100.0% [Valid] 96.5% [Ave valid] 0.145
Epoch 216 Step 81,864 ends @ 12/16/2017 12:40:38 [Train] 0.000, 100.0% [Valid] 96.1% [Ave valid] 0.147
Epoch 217 Step 82,243 ends @ 12/16/2017 12:41:50 [Train] 0.020, 100.0% [Valid] 96.3% [Ave valid] 0.152
Epoch 218 Step 82,622 ends @ 12/16/2017 12:43:05 [Train] 0.237, 92.9% [Valid] 96.2% [Ave valid] 0.152
Epoch 219 Step 83,001 ends @ 12/16/2017 12:44:21 [Train] 0.388, 92.9% [Valid] 93.9% [Ave valid] 0.162
Epoch 220 Step 83,380 ends @ 12/16/2017 12:45:33 [Train] 0.256, 92.9% [Valid] 96.0% [Ave valid] 0.148
Epoch 221 Step 83,759 ends @ 12/16/2017 12:46:49 [Train] 0.363, 92.9% [Valid] 96.3% [Ave valid] 0.151
Epoch 222 Step 84,138 ends @ 12/16/2017 12:48:03 [Train] 0.001, 100.0% [Valid] 96.3% [Ave valid] 0.152
Epoch 223 Step 84,517 ends @ 12/16/2017 12:49:18 [Train] 0.001, 100.0% [Valid] 96.5% [Ave valid] 0.149
Epoch 224 Step 84,896 ends @ 12/16/2017 12:50:31 [Train] 0.129, 92.9% [Valid]

Epoch 290 Step 109,910 ends @ 12/16/2017 14:12:19 [Train] 0.000, 100.0% [Valid] 96.5% [Ave valid] 0.151
Epoch 291 Step 110,289 ends @ 12/16/2017 14:13:31 [Train] 0.020, 100.0% [Valid] 96.6% [Ave valid] 0.150
Epoch 292 Step 110,668 ends @ 12/16/2017 14:14:47 [Train] 0.000, 100.0% [Valid] 96.4% [Ave valid] 0.151
Epoch 293 Step 111,047 ends @ 12/16/2017 14:15:59 [Train] 0.000, 100.0% [Valid] 96.6% [Ave valid] 0.148
Epoch 294 Step 111,426 ends @ 12/16/2017 14:17:15 [Train] 0.000, 100.0% [Valid] 96.1% [Ave valid] 0.149
Epoch 295 Step 111,805 ends @ 12/16/2017 14:18:30 [Train] 0.004, 100.0% [Valid] 96.5% [Ave valid] 0.143
Epoch 296 Step 112,184 ends @ 12/16/2017 14:19:43 [Train] 0.001, 100.0% [Valid] 96.5% [Ave valid] 0.147
Epoch 297 Step 112,563 ends @ 12/16/2017 14:20:59 [Train] 0.001, 100.0% [Valid] 96.4% [Ave valid] 0.146
Epoch 298 Step 112,942 ends @ 12/16/2017 14:22:11 [Train] 0.169, 100.0% [Valid] 96.6% [Ave valid] 0.150
Epoch 299 Step 113,321 ends @ 12/16/2017 14:23:27 [Train] 0.001,

KeyboardInterrupt: 